# Amazon - Baseline

### Pruebas con los distintos clasificadores para obtener los valores de transfer loss sin realizar adaptacion entre dominios

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

In [3]:
print tipo
print dataset_name
print dims
print data_path

baseline
amazon
2000
data


## Pruebas con el dataset Amazon

In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [7]:
i = 1

for tgt in domains:
    print "Entrenando dominio %d de %d" % (i, len(domains))
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "indomain_%s.pkl" % (tgt)
    model_path = os.path.join(models_path, dataset_name, model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    
    print "Score: %.3f" % svc.score(X_ts, y_ts)
    i = i+1
    
print "\nTarea terminada."

Entrenando dominio 1 de 4
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.835
Entrenando dominio 2 de 4
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.765
Entrenando dominio 3 de 4
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.802
Entrenando dominio 4 de 4
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.765

Tarea terminada.


In [8]:
df = pd.DataFrame(columns=dataframe_columns)


i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt sin adaptar
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            svc2 = load_best_score(model_path, X_tr, y_tr)
            t_error = 1-svc2.score(X_ts, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['Baseline',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1
    
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 2 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 3 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 4 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 5 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 6 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 7 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 8 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 9 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 10 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 11 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 12 de 12
Cargando modelo existente.

In [9]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,Baseline,k->d,kitchen,dvd,23.50,32.50,9.00
1,Baseline,k->e,kitchen,electronics,19.75,21.25,1.50
2,Baseline,k->b,kitchen,books,23.50,33.00,9.50
3,Baseline,d->k,dvd,kitchen,16.50,27.75,11.25
4,Baseline,d->e,dvd,electronics,19.75,28.25,8.50
5,Baseline,d->b,dvd,books,23.50,32.50,9.00
6,Baseline,e->k,electronics,kitchen,16.50,22.75,6.25
7,Baseline,e->d,electronics,dvd,23.50,33.75,10.25
8,Baseline,e->b,electronics,books,23.50,36.00,12.50
9,Baseline,b->k,books,kitchen,16.50,28.00,11.50


In [10]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/baseline/amazon.csv
Resultados guardados.
